In [3]:
import os
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
import skimage.io
import selectivesearch

In [10]:
CLASS_NAME = [
    'Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex', 'Ferrari',
    'Ford', 'Google', 'HP', 'Heineken', 'Intel', 'McDonalds', 'Mini', 'Nbc',
    'Nike', 'Pepsi', 'Porsche', 'Puma', 'RedBull', 'Sprite', 'Starbucks',
    'Texaco', 'Unicef', 'Vodafone', 'Yahoo', 'Background'
]
annot_train = np.loadtxt(os.path.join('flickr_logos_27_dataset', 'flickr_logos_27_dataset_training_set_annotation.txt'), dtype='a')
print('train_annotation: {}'.format(annot_train.shape[0]))

train_annotation: 2


In [15]:
def parse_annot(annot):
    fn = annot[0].decode('utf-8')
    class_name = annot[1].decode('utf-8')
    train_subset_class = annot[2].decode('utf-8')
    return fn, class_name, train_subset_class


def iou(obj_proposal, annot_rect):
    """

    Arguments:
    obj_proposals -- rectangles of object proposals with coordinates (x, y, w, h)
    annot_rect -- rectangle of ground truth with coordinates (x1, y1, x2, y2)
    """
    xi1 = max(obj_proposal[0], annot_rect[0])
    yi1 = max(obj_proposal[1], annot_rect[1])
    xi2 = min(obj_proposal[0] + obj_proposal[2], annot_rect[2])
    yi2 = min(obj_proposal[1] + obj_proposal[3], annot_rect[3])
    inter_area = (yi2 - yi1) * (xi2 - xi1)

    # Calculate the union area by using formula: union(A, B) = A + B - inter_area
    box1_area = obj_proposal[2] * obj_proposal[3]
    box2_area = (annot_rect[2] - annot_rect[0]) * (
        annot_rect[3] - annot_rect[1])
    union_area = box1_area + box2_area - inter_area

    # Compute the IoU
    iou = inter_area / union_area

    return iou

def get_bg_proposals(object_proposals, annot):
    annot_rect = np.array(list(map(lambda x: int(x), annot[3:])))
    bg_proposals = []
    for obj_proposal in object_proposals:
        if iou(obj_proposal, annot_rect) <= 0.5:
            bg_proposals.append(obj_proposal)
    return bg_proposals

def get_object_proposals(img, scale=500, sigma=0.9, min_size=10):
    # Selective search
    img_lbl, regions = selectivesearch.selective_search(
        img, scale=scale, sigma=sigma, min_size=min_size)

    candidates = set()
    for r in regions:
        # excluding same rectangle (with different segments)
        if r['rect'] in candidates:
            continue
        # excluding regions smaller than 500 pixels
        x, y, w, h = r['rect']
        if r['size'] < 2000 or w > 0.95 * img.shape[1] or h > 0.95 * img.shape[0]:
            continue
        # excluding the zero-width or zero-height box
        if r['rect'][2] == 0 or r['rect'][3] == 0:
            continue
        # distorted rects
        if w / h > 5 or h / w > 5:
            continue
        candidates.add(r['rect'])

    return candidates

def gen_annot_file_line(img_fn, class_name, train_subset_class, rect):
    rect = ' '.join(map(str, rect))
    line = ' '.join([img_fn, class_name, train_subset_class, rect])
    return line


def gen_annot_file_lines(annot):
    lines = []
    # Get original annot line
    img_fn, class_name, train_subset_class = parse_annot(annot)
    annot_rect = np.array(list(map(lambda x: int(x), annot[3:])))
    lines.append(
        gen_annot_file_line(img_fn, class_name, train_subset_class,
                            annot_rect))

    # Load image
#     img = skimage.io.imread(os.path.join(os.path.join('flickr_logos_27_dataset', 'flickr_logos_27_dataset_images'), img_fn))

#     # Selective search
#     object_proposals = get_object_proposals(img)
#     if len(object_proposals) == 0:
#         return lines

#     # Background proposals
#     bg_proposals = get_bg_proposals(object_proposals, annot)
#     if len(bg_proposals) == 0:
#         return lines

#     # Select bg proposal
#     bg_proposal = bg_proposals[np.random.choice(
#         np.array(bg_proposals).shape[0])]
#     x1, y1, x2, y2 = bg_proposal[0], bg_proposal[
#         1], bg_proposal[0] + bg_proposal[2], bg_proposal[1] + bg_proposal[3]
#     lines.append(
#         gen_annot_file_line(img_fn, CLASS_NAME[-1], train_subset_class,
#                             [x1, y1, x2, y2]))

    print(lines)
    return lines

In [16]:
futures_all = []
n_workers = os.cpu_count()
with ProcessPoolExecutor(n_workers) as executer, open(os.path.join('flickr_logos_27_dataset', 'train_annot_with_bg_class.txt'), 'w') as fw:
    for annot in annot_train:
        futures_all.append(executer.submit(gen_annot_file_lines(annot)))
    for future_one in as_completed(futures_all):
        print('\n'.join(future_one.result()))
        fw.writelines('\n'.join(future_one.result()))
        fw.writelines('\n')

['144503924.jpg Adidas 1 38 12 234 142']
['2451569770.jpg Adidas 1 242 208 413 331']


TypeError: 'list' object is not callable

In [15]:
annot_train

array([[b'144503924.jpg', b'Adidas', b'1', ..., b'12', b'234', b'142'],
       [b'2451569770.jpg', b'Adidas', b'1', ..., b'208', b'413', b'331'],
       [b'390321909.jpg', b'Adidas', b'1', ..., b'5', b'89', b'60'],
       ...,
       [b'2472817996.jpg', b'Yahoo', b'6', ..., b'4', b'499', b'106'],
       [b'2514220918.jpg', b'Yahoo', b'6', ..., b'69', b'342', b'157'],
       [b'386891249.jpg', b'Yahoo', b'6', ..., b'10', b'310', b'49']],
      dtype='|S14')